 ## 9.1 ROOM - podstawy

Przyjrzyjmy się `ROOM` - jest to warstwa abstrakcji służąca jako punkt dostępu do do bazy `SQLite`, znacznie upraszcza pracę z wewnętrzną bazą na urzadzeniu. W tej aplikacji zaimplementujemy bazę przechowującą słowa - umożliwimy wyświetlanie wszystkich słów oraz dodanie nowego elementu. Wykorzystamy elementy architektury - `ViewModel`, `LiveData` oraz `ROOM`

Rozpocznijmy od dodania odpowiednich zależności

In [ ]:
// ROOM
implementation "androidx.room:room-runtime:2.4.3"
annotationProcessor "androidx.room:room-compiler:2.4.3"

// ViewModel
implementation 'androidx.lifecycle:lifecycle-viewmodel:2.5.1'
// LiveData
implementation "androidx.lifecycle:lifecycle-livedata:2.5.1"

Nasza aplikacja składała się tylko z jednej aktywności, na której dodamy `RecyclerView` - tym razem wykorzystamy `ListAdapter` (zamiast zwykłego adaptera) oraz `DiffUtil` do określenia różnic między elementami kolekcji.

### **ROOM**

#### **Entity**

Tworzenie bazy rozpoczniemy od klasy `Word` jest to byt (`Entity`) reprezentujący tabelę w bazie `SQLite`.

In [ ]:
@Entity(tableName = "word_table")
public class Word {

`@Entity(tableName = "word_table")` - każda klasa oznaczona adnotacją `@Entity` reprezentuje tabelę `SQLite`. Możemy określić nazwę tabeli jaką chcemy nadać w samej bazie wykorzystując właściwość `tableName` - tylko w przypadku gdy chcemy aby nazwa tabeli była inna niż nazwa klasy.

Nasza klasa będzie posiadała jedną właściwość (pole), będzie to słowo (`word`) typu `String`. Każda właściwość w klasie reprezentuje kolumnę w tabeli - więc w naszym przykładzie zostanie utworzona jedna tabela o nazwie `word_table` posiadająca jedną kolumnę o nazwie `word`.

In [ ]:
    @PrimaryKey
    @NonNull
    @ColumnInfo(name = "word")
    private final String word;

- `@PrimaryKey` - każda `Entity` wymaga posiadania jednego klucza głównego (unikalnego identyfikatora), w tym przykładzie, dla uproszczenia, samo słowo będzie jednocześnie kluczem głównym.
- `@NonNull` - oznacza że parametr, pole lub wartość zwracana metody nie może mieć wartości `null`
- `@ColumnInfo(name = "word")` - pozwala nadać nazwę kolumnie, jeżeli jest ona inna niż nazwa pola - w tym przykładzie jest ona taka sama, więc tą adnotację możemy usunąć.

Do naszej klasy dodajmy jeszcze getter oraz konstruktor

In [ ]:
    public Word(@NonNull String word) {this.word = word;}

    public String getWord(){return this.word;}
}

#### **DAO**

`DAO` (*Data Access Object*) weryfikuje kod `SQL` podczas kompilacji i powiązuje go z odpowiednią metodą - musi być interfejsem lub klasą abstrakcyjną.

In [ ]:
@Dao
public interface WordDao {

Podobnie jak przy `@Entity` posługujemy się adnotacją `@Dao` do oznaczenia interfejsu jako części `ROOM`. Tutaj dodamy kilka metod pozwialających nam dodać element do bazy, usunąć wszystkie elementy oraz wyciągnąć z bazy listę posortowanych elementów. Rozpocznijmy od metody `insert`

In [ ]:
    @Insert(onConflict = OnConflictStrategy.IGNORE)
    void insert(Word word);

- `void insert(Word word);` deklaruje metodę wstawiającą jeden element
- `@Insert` - specjalna adnotacja, która nie wymaga podawania wprost kodu `SQL` - istanieją jeszcze `@Delete` oraz `@Update` - mocno ułatwia to wykonywanie podstawowych operacji na bazie
- `onConflictStrategy` - określa zachowanie w przypadku wystąpienia konfliktu podczas próby dodania elementu do bazy (czyli próby dodania elementu który już istnieje) - tutaj będziemy ignorować, czyli próba dodania słowa które już istnieje w bazie zostanie zignorowana

Kolejną będzie metoda usuwająca wszystkie elementy z listy

In [ ]:
    @Query("DELETE FROM word_table")
    void deleteAll();

Adnotacja `@Delete` służy do usuwania tylko jednego elementu, do usunięcia wszystkich musimy dodać odpowiednie zapytanie.
- `@Query` - wymaga podania zapytania SQL jako parametru typu `String`

Ostatnią metodą będzie `getAlphabetizedWords` zwracająca posortowaną listę wszystkich słów.

In [ ]:
    @Query("SELECT * FROM word_table ORDER BY word ASC")
    LiveData<List<Word>> getAlphabetizedWords();

Metoda zwraca listę wszystkich słów jako `LiveData`

#### **RoomDatabase**

Jedną z ważniejszych cech bazy `ROOM` jest brak zezwolenia na wykonywanie zapytań na wątku głównym (nazywanym również wątkiem ui). Wszystkie zapytania zwracające dane z bazy są wykonywane asynchronicznie. Klasa reprezentująca samą bazę musi być **klasą abstrakcyjną** i musi rozszerzać klasę `RoomDatabase`.

In [ ]:
@Database(entities = {Word.class}, version = 1, exportSchema = false)
public abstract class WordRoomDatabase extends RoomDatabase {

Ponownie wykorzystujemy adnotację aby poinformować kompilator że ta klasa jest komponentem `ROOM`. Sama adnotacja `@Database` przyjmuje kilka parametrów
- `entities` - określa byty (tabele) należące do bazy
- `version` - określa wersję bazy
- `exportSchema` - migracje baz danych tutaj pominiemy, więc ustawimy na `false`

Baza wystawia `DAO` przez abstrakcyjny getter dla każdego `@Dao`.

In [ ]:
    public abstract WordDao wordDao();

Samą bazę zaimplementujemy jako `Singleton` - będzie to implementacje [**double-checked**](https://www.baeldung.com/java-singleton-double-checked-locking) - aby zapobiec tworzeniu wielu instancji bazy. Innymi słowy chcemy zapobiec wielokrotnemu otwarciu bazy i niekontrolowanemu wykonywaniu operacji.

W pierwszym kroku dodajmy zmienną `INSTANCE` w której będziemy przechowywać naszą jedyną instancję teggo obiektu

In [ ]:
    private static volatile WordRoomDatabase INSTANCE;

Tworzymy `ExecutorService`, który wykorzystamy do wykonywania operacji asynchronicznie.

In [ ]:
    private static final int NUMBER_OF_THREADS = 4;
    public static final ExecutorService databaseWriteExecutor =
            Executors.newFixedThreadPool(NUMBER_OF_THREADS);

Potrzebujemy jeszcze metodę `getDatabase`, która zainicjuje nam bazę jeżeli ta jeszcze nie istnieje, lub zwróci nam `INSTANCE` jeżeli została utworzona wcześniej. Wpierw wykonajmy podwójne sprawdzenie oraz dodajmy blok `synchronized`

In [ ]:
    public static WordRoomDatabase getDatabase(final Context context) {
        if (INSTANCE == null) {
            synchronized (WordRoomDatabase.class) {
                if (INSTANCE == null) {
                    
                }
            }
        }
        return INSTANCE;
    }

Wewnątrz bloku synchronized tworzymy bazę jeżeli `INSTANCE == null`

In [ ]:
    public static WordRoomDatabase getDatabase(final Context context) {
        if (INSTANCE == null) {
            synchronized (WordRoomDatabase.class) {
                if (INSTANCE == null) {
                    INSTANCE = Room.databaseBuilder(context.getApplicationContext(),
                                    WordRoomDatabase.class, "word_database_javav2")
                            .build();
                }
            }
        }
        return INSTANCE;
    }

W tym celu wykorzystujemy `Room.databaseBuilder` który przyjmuje kontekst. Zwróćmy uwagę na zastosowanie kontekstu aplikacji, a nie np. aktywności - kontekst aplikacji jest najszerszym kontekstem i najczęściej to na nim tworzymy bazę i wszystkie powiązane z nią elementy.

### **ViewModel**

Utwórzmy `ViewModel` - tym razem rozszerzymy `AndroidViewModel`, a nie jak poprzednio `ViewModel` - różnica jest w parametrze. `AndroidViewModel` może przekazać parametr. Dodajmy też dwa pola reprezentujące bazę danych oraz `LiveData` naszej listy słów.

In [ ]:
public class WordViewModel extends AndroidViewModel {

    private final WordRoomDatabase db;

    private final LiveData<List<Word>> allWords;

Ponieważ `RoomDatabase` wymaga podania kontekstu (zazwyczaj jest nim `Application`)

In [ ]:
    public WordViewModel (Application application) {
        super(application);
        db = WordRoomDatabase.getDatabase(application);
        allWords = db.wordDao().getAlphabetizedWords();
    }

Dodajmy również getter

In [ ]:
    public LiveData<List<Word>> getAllWords() { return allWords; }

Ostatnią metodą klasy będzie `insert` umożliwiająca dodanie elementu do bazy - wykorzystamy `ExecutorService` zdefiniowany w klasie `WordRoomDatabase`

In [ ]:
    public void insert(Word word) {
        WordRoomDatabase.databaseWriteExecutor.execute(() -> 
            db.wordDao().insert(word));
    }

### **RecyclerView + ListAdapter + DiffUtil**

Przejdźmy do wyświetlenia danych i umożliwienia dodania nowego słowa. Wpierw zdefiniujmy layouty. Layout głównej aktywności zawierający `RecyclerView`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout
    xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".view.MainActivity">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/recyclerview"
        android:layout_width="0dp"
        android:layout_height="0dp"
        tools:listitem="@layout/recyclerview_item"
        android:padding="12dp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintLeft_toLeftOf="parent"
        app:layout_constraintRight_toRightOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <com.google.android.material.floatingactionbutton.FloatingActionButton
        android:id="@+id/fab"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_margin="16dp"
        android:contentDescription="add word"
        android:src="@drawable/ic_add"/>

</androidx.constraintlayout.widget.ConstraintLayout>

Dodajmy również layout dla pojedynczego elementu listy

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    android:orientation="vertical"
    android:layout_width="match_parent"
    android:layout_height="wrap_content">

    <TextView
        android:id="@+id/textView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content" />
</LinearLayout>

Rozpocznijmy od utworzenia klasy `ViewHolder`

In [ ]:
class WordViewHolder extends RecyclerView.ViewHolder {
    private final TextView wordItemView;

    private WordViewHolder(View itemView) {
        super(itemView);
        wordItemView = itemView.findViewById(R.id.textView);
    }

    public void bind(String text) {
        wordItemView.setText(text);
    }

    static WordViewHolder create(ViewGroup parent) {
        return new WordViewHolder(LayoutInflater.from(parent.getContext())
                .inflate(R.layout.recyclerview_item, parent, false));
    }
}

Wykorzystamy tutaj metodę statyczną `create` z prywatnym konstruktorem. Następnie Dodajmy klasę `WordComparator` rozszerzjącą klasę `DiffUtil.ItemCallback`, służy ona dwóm celom
- indeksowanie list
- rozróżnianie elementów

Mamy do zaimplementowania dwie metody
- `areItemsTheSame` - wskazująca czy dwa elementy są tym samym (np. porównanie referencyjne)
- `areContentsTheSame` - wskazująca czy dwa elementy są takie same (np. porównanie strukturalne)

In [ ]:
class WordsComparator extends DiffUtil.ItemCallback<Word> {

    @Override
    public boolean areItemsTheSame(@NonNull Word oldItem, @NonNull Word newItem) {
        return oldItem == newItem;
    }

    @Override
    public boolean areContentsTheSame(@NonNull Word oldItem, @NonNull Word newItem) {
        return oldItem.getWord().equals(newItem.getWord());
    }
}

Pozostaje implementacja `WordListAdapter` rozszerzającej `ListAdapter` - jest to klasa dostarczająca taką samą funkcjonalność jak `RecyclerView.Adapter`, dodatkowo umożliwia obliczenie różnic między listami (na wątku roboczym)

In [ ]:
public class WordListAdapter extends ListAdapter<Word, WordViewHolder> {

    public WordListAdapter(@NonNull DiffUtil.ItemCallback<Word> diffCallback) {
        super(diffCallback);
    }

    @NonNull
    @Override
    public WordViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        return WordViewHolder.create(parent);
    }

    @Override
    public void onBindViewHolder(WordViewHolder holder, int position) {
        Word current = getItem(position);
        holder.bind(current.getWord());
    }
}

### **Aktywność**

Do głównej aktywności dodajmy dwie zmienne

In [ ]:
    private WordViewModel wordViewModel;
    private int num = 0; // licznik słów

W metodzie `onCreate` tworzymy `wordViewModel` przez wykorzystanie `ViewModelProvider`

In [ ]:
wordViewModel = new ViewModelProvider(this).get(WordViewModel.class);

Tutaj zaznaczmy że nie jest to najlepsza metoda, ale w tym prostym przykładzie będzie wystarczająca

Dodajmy `Recyclerview`

In [ ]:
RecyclerView recyclerView = findViewById(R.id.recyclerview);
final WordListAdapter adapter = new WordListAdapter(new WordsComparator());
recyclerView.setAdapter(adapter);
recyclerView.setLayoutManager(new LinearLayoutManager(this));

Podłączmy obserwator do listy słów

In [ ]:
wordViewModel.getAllWords().observe(this, adapter::submitList);

Gdy pojawia się nowa wersja listy wywołujemy metodę `submitList` klasy `ListAdapter` - lista zostaje automatycznie odświeżona przy każdej zmianie.

Na koniec dodajmy możliwość dodania nowego słowa

In [ ]:
FloatingActionButton fab = findViewById(R.id.fab);
fab.setOnClickListener(v -> {
                wordViewModel.insert(new Word("word " + num));
                num++;
            });

Możemy przetestować aplikację

<img src="https://media3.giphy.com/media/MMcJ5i46Q0FIBspUEQ/giphy.gif" width="150" />